# 1. FER2013 XGBoost 분류

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## 1.1. 학습에 사용될 하이퍼 패러미터 선언 및 Random Seed 설정

In [2]:
np.random.seed(201)

In [3]:
FER2013_DIR = "datasets/fer2013/prep/"
INPUT_TRAIN_DATA = 'fer2013_X.npy'
LABEL_TRAIN_DATA = 'fer2013_y.npy'

TEST_SPLIT = 0.2
RND_SEED = 0

In [4]:
input_data = np.load(open(FER2013_DIR + INPUT_TRAIN_DATA, 'rb'))
label_data = np.load(open(FER2013_DIR + LABEL_TRAIN_DATA, 'rb'))

In [5]:
nsamples, nx, ny, nc = input_data.shape
input_data = input_data.reshape((nsamples,nx*ny*nc))

In [6]:
input_data.shape

(12045, 2304)

In [7]:
train_input, eval_input, train_label, eval_label = train_test_split(input_data,
                                                                    label_data,
                                                                    test_size=TEST_SPLIT,
                                                                    random_state=RND_SEED)

## 1.2 XGBoost 모델을 이용하여 학습

In [8]:
train_data = xgb.DMatrix(train_input, label=train_label)

data_list = [(train_data, 'train')]

In [9]:
clf = OneVsRestClassifier(XGBClassifier(n_estimators=5000,
                                        booster='gbtree',
                                        tree_method ='gpu_hist',
                                        predictor = 'gpu_predictor'), -1)
clf.fit(train_input, train_label)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster='gbtree',
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=5000, n_jobs=None,
                                            num_parallel_tree=None,
                                            objective='binary:logistic',
                                       

정확도

In [10]:
clf.score(eval_input, eval_label)

0.7638024076380241

## 1.3 K-fold  Cross Validation 이용

In [11]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, random_state=RND_SEED, shuffle=True)

for train_index, test_index in kf.split(input_data):
    X_train, X_test = input_data[train_index], input_data[test_index]
    y_train, y_test = label_data[train_index], label_data[test_index]

In [13]:
clf = OneVsRestClassifier(XGBClassifier(n_estimators=5000,
                                        booster='gbtree',
                                        tree_method ='gpu_hist',
                                        predictor = 'gpu_predictor'), -1)
clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster='gbtree',
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=5000, n_jobs=None,
                                            num_parallel_tree=None,
                                            objective='binary:logistic',
                                       

정확도

In [14]:
clf.score(X_test, y_test)

0.7699335548172758